In [5]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


interbankExposure = pd.read_csv('interbankExposures.csv', header=None)
bankEquities = pd.read_csv('bankEquities.csv', header=None)
bankAsset = pd.read_csv('bankAssetWeightedNetwork.csv', header=None)

## Calculate the losses due to counterparty default risk:

In [9]:
def furfine(interbank_exposure, bank_equities, recovery_rate=1.0):
    n_banks = len(bank_equities)
    losses = np.zeros(n_banks)
    defaulting_banks = []

    for i in range(n_banks):
        if bank_equities.iloc[i, 0] <= 0:
            defaulting_banks.append(i)

    while defaulting_banks:
        bank = defaulting_banks.pop()
        for counterparty, exposure in enumerate(interbank_exposure.iloc[:, bank]):
            if exposure > 0 and counterparty not in defaulting_banks:
                losses[counterparty] += exposure * (1 - recovery_rate)
                bank_equities.iloc[counterparty, 0] -= exposure * (1 - recovery_rate)

                if bank_equities.iloc[counterparty, 0] <= 0 and counterparty not in defaulting_banks:
                    defaulting_banks.append(counterparty)

    return losses


In [11]:
# print(bankEquities.shape)
# print(bankEquities.head())


(1, 145)
        0       1      2           3         4         5          6    \
0  465710.0  4436.7  13159  16229000.0  438420.0  271300.0  7572800.0   

          7      8         9    ...    135         136       137       138  \
0  14267000.0  99580  178320.0  ...  32812  32123000.0  699090.0  515310.0   

     139         140         141       142       143         144  
0  14259  11382000.0  14033000.0  129430.0  874880.0  26855000.0  

[1 rows x 145 columns]


In [12]:
bankEquities = bankEquities.transpose()


In [25]:
shocked_bank_equities = bankEquities.copy()

# Apply an initial exogenous shock (e.g., 20% loss) to a specific bank (e.g., bank 5)
shocked_bank = 5
shock_magnitude = 0.2
shocked_bank_equities.iat[shocked_bank, 0] *= (1 - shock_magnitude)


In [14]:
recovery_rate = 0.5  # Example value for the recovery rate
losses = furfine(interbankExposure, shocked_bank_equities, recovery_rate)
print("Losses due to counterparty default risk (Furfine algorithm):")
print(losses)


Losses due to counterparty default risk (Furfine algorithm):
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


losses appear to be zero, This can happen if the initial exogenous shock is not large enough to cause the affected bank to default, or if there is no significant interbank exposure between the affected bank and other banks in the network.

check the interbank exposures to see if there are significant exposures

In [15]:
print(interbankExposure.head())


       0    1    2        3        4         5          6          7    \
0      0.0  0.0  0.0      0.0      0.0       0.0        0.0        0.0   
1      0.0  0.0  0.0      0.0      0.0       0.0        0.0        0.0   
2      0.0  0.0  0.0      0.0      0.0       0.0        0.0        0.0   
3  96450.0  0.0  0.0      0.0  97484.0  149400.0  1192900.0  7327900.0   
4      0.0  0.0  0.0  52808.0      0.0       0.0     9576.7        0.0   

       8        9    ...  135        136      137       138  139        140  \
0      0.0      0.0  ...  0.0        0.0      0.0       0.0  0.0        0.0   
1      0.0      0.0  ...  0.0        0.0      0.0       0.0  0.0        0.0   
2      0.0      0.0  ...  0.0        0.0      0.0       0.0  0.0        0.0   
3  89233.0  76598.0  ...  0.0  6388900.0  38878.0  107500.0  0.0  5451800.0   
4      0.0      0.0  ...  0.0    51289.0      0.0       0.0  0.0    43767.0   

         141      142       143        144  
0        0.0      0.0       0.0    

try updating the shocks to see if the losses increase

In [44]:
shocked_bank = 5  # Changed this to another bank index
shock_magnitude = 1 # Changed this to a larger shock magnitude 
shocked_bank_equities = bankEquities.copy()
shocked_bank_equities.iat[shocked_bank, 0] *= (1 - shock_magnitude)


In [ ]:
recovery_rate = 0.4  # Try different recovery rate values
losses = furfine(interbankExposure, shocked_bank_equities, recovery_rate)
print("Losses due to counterparty default risk (Furfine algorithm):")
print(losses)
